In [2]:
import csv
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import math

In [5]:
#general model
professionChoice = 'Doctor'
days_in_future = 10
time_step = []
cases = []
model = load_model(f'../models/profession/lstm-model-{professionChoice}-.h5')

window_size = 5
split_time = 10

def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

with open(f'../final profession csvs/{professionChoice}.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  steps = 0
  for row in reader:
    cases.append(float(row[2]))
    time_step.append(steps)
    steps+=1

for i in range(days_in_future):
    time_step.append(time_step[-1] + 1)

for i in range(days_in_future):
    cases.append(0)

series = np.array(cases)
time = np.array(time_step)

rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

prediction = math.ceil(rnn_forecast[-1])

if prediction == 1:
    print(math.ceil(rnn_forecast[-1]), "case is predicted for " + str(days_in_future) + f" days in future for the profession of {professionChoice}")
else:
    print(math.ceil(rnn_forecast[-1]), "cases are predicted for " + str(days_in_future) + f" days in future for the profession of {professionChoice}")

5 cases are predicted for 10 days in future for the profession of Doctor
